In [79]:
import requests
import json

# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy

import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import cv2
import matplotlib.pyplot as plt
import datetime as dt

%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

devices = {'Streetscape': ('SWLSANDBOX1', 'img/Streetscape.png'), 'Outside': ('SWLSANDBOX3', 'img/Outside.png'),
           'UnderRaincoat': ('SWLSANDBOX2', 'img/UnderRaincoat.png')}

events = {'City Moments': (dt.datetime(2019, 8, 16, 19), dt.datetime(2019, 8, 17, 2)),
         'Open Sidewalk #4': (dt.datetime(2019, 3, 2, 15), dt.datetime(2019, 3, 2, 19)),
         'Sidewalk Summer Open House': (dt.datetime(2019, 6, 29, 12), dt.datetime(2019, 6, 29, 18)),
         'Startup Open House': (dt.datetime(2019, 9, 26, 16), dt.datetime(2019, 9, 26, 20)),
         'Tap:Ex Augmented Opera': (dt.datetime(2019, 11, 20, 18), dt.datetime(2019, 11, 23, 22))}

selectedEvent = events['City Moments']

In [80]:
def eventSelector(event):
    global selectedEvent
    if event == 'None':
        selectedEvent = 'None'
    else:
        selectedEvent = events[event]

eventNames = list(events.keys())
eventNames.append('None')

interact_manual(eventSelector, event=widgets.Dropdown(options=eventNames))


interactive(children=(Dropdown(description='event', options=('City Moments', 'Open Sidewalk #4', 'Sidewalk Sum…

<function __main__.eventSelector(event)>

In [83]:
eventCounts = pd.DataFrame(columns=['event', 'count'])

for event in events.keys():
    startdate = events[event][0]
    enddate = events[event][1]
    
    eventCountsQuery = """
    query {
      feedCountMetrics(
        serialnos:["SWLSANDBOX1", "SWLSANDBOX2", "SWLSANDBOX3"],
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York",
        interval:"24h") {
        edges {
          node {
            serialno
            result
            objClass
            time
          }
        }
      }
    }
    """ % (startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))
    
    response = requests.post(url, json={'query': eventCountsQuery}, headers = {'Authorization':token}).json()

    count = 0
    for edge in response['data']['feedCountMetrics']['edges']:
        count = count + edge['node']['result']
    
    this_event = pd.DataFrame([[event, count]], columns=['event', 'count'])
    eventCounts = eventCounts.append(this_event)
    
eventCounts = eventCounts.reset_index(drop=True)
eventCounts
    

,event,count
0,City Moments,257.0
1,Open Sidewalk #4,4975.0
2,Sidewalk Summer Open House,19533.0
3,Startup Open House,3748.0
4,Tap:Ex Augmented Opera,8556.0


In [86]:
def dwellTimeEventsBar(startdate, enddate):
    
    dwellTimeEventsQuery = """
    query {
      feedCountMetrics(
        serialnos:["SWLSANDBOX1", "SWLSANDBOX2", "SWLSANDBOX3"],
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York",
        interval:"24h") {
        edges {
          node {
            serialno
            result
            objClass
            time
          }
        }
      }
    }
    """ % (startdate.strftime("%Y-%m-%dT00:00:00"), enddate.strftime("%Y-%m-%dT00:00:00"))    
    
    response = requests.post(url, json={'query': dwellTimeEventsQuery}, headers = {'Authorization':token}).json()

    count = 0
    for edge in response['data']['feedCountMetrics']['edges']:
        count = count + edge['node']['result']
        
    this_date = pd.DataFrame([['Selected Timeframe', count]], columns=['event', 'count'])
    
    eventCounts_copy = eventCounts
    eventCounts_copy = eventCounts_copy.append(this_date)
    
    plt.barh(eventCounts_copy['event'], eventCounts_copy['count'])

interact_manual(dwellTimeEventsBar, startdate=widgets.DatePicker(value=pd.to_datetime('2019-01-01')), 
                enddate=widgets.DatePicker(value=pd.to_datetime('2020-01-01')))

interactive(children=(DatePicker(value=Timestamp('2019-01-01 00:00:00'), description='startdate'), DatePicker(…

<function __main__.dwellTimeEventsBar(startdate, enddate)>

In [2]:
def getHeatmapImage(device, startdate, enddate):
    try:
        serial = devices[device][0]
    except KeyError:
        raise Exception('Invalid device name.')
    
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))

    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
    
    
    img = cv2.imread(devices[device][1])
    heatmap = cv2.resize(heatarr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    combined_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (15, 10))
    plt.axis('off')
    plt.imshow(combined_image)

In [3]:
interact_manual(getHeatmapImage, device=widgets.Dropdown(options=list(devices.keys())), 
                    startdate=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
             enddate=widgets.DatePicker(value=pd.to_datetime('2020-01-01')))

interactive(children=(Dropdown(description='device', options=('Streetscape', 'Outside', 'UnderRaincoat'), valu…

<function __main__.getHeatmapImage(device, startdate, enddate)>

In [4]:
def getcountData(device, eventname):
    try:
        serial = devices[device][0]
        eventstart = events[eventname][0]
        eventend = events[eventname][1]
    except KeyError:
        raise Exception('Invalid device or event name.')

    query = """
    query {
      feedCountMetrics(
        serialnos: ["%s"],
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York",
        interval:"1h") {
        edges {
          node {
            serialno
            result
            objClass
            time
          }
        }
      }
    }
    """ % (serial, eventstart.strftime("%Y-%m-%dT%H:00:00"), eventend.strftime("%Y-%m-%dT%H:00:00"))
    
    countdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    return countdata.json()

In [5]:
y = getcountData('Outside', 'City Moments')

In [6]:
y['data']['feedCountMetrics']['edges']

[{'node': {'objClass': 'pedestrian',
   'result': 10.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-16T19:00:00-04:00'}},
 {'node': {'objClass': 'pedestrian',
   'result': 4.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-16T20:00:00-04:00'}},
 {'node': {'objClass': 'pedestrian',
   'result': 0.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-16T21:00:00-04:00'}},
 {'node': {'objClass': 'pedestrian',
   'result': 1.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-16T22:00:00-04:00'}},
 {'node': {'objClass': 'pedestrian',
   'result': 0.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-16T23:00:00-04:00'}},
 {'node': {'objClass': 'pedestrian',
   'result': 0.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-17T00:00:00-04:00'}},
 {'node': {'objClass': 'pedestrian',
   'result': 0.0,
   'serialno': 'SWLSANDBOX3',
   'time': '2019-08-17T01:00:00-04:00'}}]